In [ ]:
# https://www.jstatsoft.org/article/view/v035c02/v35c02.pdf

In [1]:
import pandas as pd
import pyper as pr

In [2]:
# CREATE A R INSTANCE WITH PYPER:    
r = pr.R()

In [3]:
import pandas as pd
DF = pd.read_csv('AMR_NotPaid.csv')
DF.shape
DF.head()

,nb
0,5500
1,6513
2,7602
3,8871
4,9828


In [4]:
# PASS DATA FROM PYTHON TO R (DF gets stored as rDF:
r.assign("rDF", DF["nb"]) 

In [5]:
print (r("summary(rDF)"))

try({summary(rDF)})
       nb       
 Min.   : 2493  
 1st Qu.: 6993  
 Median : 8567  
 Mean   : 8927  
 3rd Qu.:10672  
 Max.   :16811  



In [6]:
print (r("str(rDF)"))

try({str(rDF)})
'data.frame':	1019 obs. of  1 variable:
 $ nb: int  5500 6513 7602 8871 9828 10103 10064 9559 8881 9820 ...



In [7]:
# load R package
r("library(forecast)")

'try({library(forecast)})\n'

In [ ]:
## R snippet to (1) find arima params, (2) fit a arima model, (3) forecast, (4) output a csv
'''
## Auto.Arima
rnb_arima <- auto.arima(DF[,9])  # ALWAYS INPUT RAW DATA THAT IS 'NOT' A ts() object
arimaorder(rnb_arima)
## STLM - apply Auto.Arima model to a ts object
# Make the 'y' variables to a ts object
rnb <- ts(DF$y, start=2016,freq=365)
fit <- stlm(rnb, modelfunction=Arima, order=arimaorder(rnb_arima))
fit_arima_fc <- forecast(fit, h=442)
write.csv(fit_arima_fc, "6_rnb_arima_pred_fc.csv")
'''

In [36]:
r("rnb_arima <- auto.arima(rDF$nb, seasonal = T)")

'try({rnb_arima <- auto.arima(rDF$nb, seasonal = T)})\n'

In [37]:
print(r("arimaorder(rnb_arima)"))

try({arimaorder(rnb_arima)})
p d q 
2 1 2 



In [38]:
# convert data frame into a ts object
r("rnb <- ts(rDF$nb, start=2016,freq=365)")

'try({rnb <- ts(rDF$nb, start=2016,freq=365)})\n'

In [39]:
r("fit <- stlm(rnb, modelfunction=Arima, order=arimaorder(rnb_arima))")

'try({fit <- stlm(rnb, modelfunction=Arima, order=arimaorder(rnb_arima))})\n'

In [40]:
print(r("fit"))

try({fit})
$stl
Time Series:
Start = c(2016, 1) 
End = c(2018, 289) 
Frequency = 365 
          Data     Trend   Seasonal365    Remainder
2016.000  5500  5499.132 -1419.5357821  1420.403614
2016.003  6513  5507.202  -131.0510275  1136.848680
2016.005  7602  5515.273  1656.0414709   430.686002
2016.008  8871  5523.343  2648.8050071   698.852286
2016.011  9828  5531.413  3868.0789379   428.508175
2016.014 10103  5539.483  3392.0591061  1171.457827
2016.016 10064  5547.553  3233.9910271  1282.455727
2016.019  9559  5555.623  3900.7086130   102.667961
2016.022  8881  5563.694  3852.8874416  -535.581047
2016.025  9820  5571.764  4600.2501277  -352.013913
2016.027 10494  5579.834  4952.6132856   -38.447251
2016.030 10273  5587.904  4068.5049749   616.590880
2016.033 10131  5595.974  4003.7461777   531.279498
2016.036  9925  5604.045  3423.0033366   897.952159
2016.038  9306  5612.115  3882.1169157  -188.231600
2016.041  8485  5620.185  3952.6392429 -1087.824107
2016.044  9289  5628.255  4286

In [13]:
r("fit_arima_fc <- forecast(fit, h=442)")

'try({fit_arima_fc <- forecast(fit, h=442)})\n'

In [14]:
print(r("(fit_arima_fc)"))

try({(fit_arima_fc)})
          Point Forecast      Lo 80     Hi 80      Lo 95     Hi 95
2018.7918       8140.240  7314.1894  8966.290  6876.9048  9403.575
2018.7945       8216.269  7275.9856  9156.553  6778.2296  9654.309
2018.7973       8028.624  7088.0127  8969.235  6590.0833  9467.164
2018.8000       7358.646  6402.1265  8315.167  5895.7753  8821.518
2018.8027       7254.012  6291.9539  8216.071  5782.6708  8725.354
2018.8055       6543.884  5576.4100  7511.359  5064.2600  8023.509
2018.8082       7136.864  6131.9073  8141.822  5599.9152  8673.814
2018.8110       7713.711  6660.4596  8766.962  6102.9020  9324.520
2018.8137       8025.097  6942.7905  9107.404  6369.8520  9680.342
2018.8164       8423.395  7330.6341  9516.156  6752.1615 10094.628
2018.8192       7825.418  6728.8844  8921.951  6148.4147  9502.421
2018.8219       6544.041  5443.6058  7644.475  4861.0708  8227.010
2018.8247       6647.052  5538.5043  7755.600  4951.6744  8342.430
2018.8274       6857.965  5735.4946  798

In [15]:
r("df_mape <- data.frame(rnb, fit_arima_fc$fitted)") 
r("mape_arima <- mean(abs((df_mape$rnb - df_mape$fit_arima_fc.fitted)/df_mape$rnb))") 
print(r("mape_arima"))

try({mape_arima})
[1] 0.05488735



In [23]:
r("df_rnb <- c(rnb, rep(NA,442))") 
r("df_arima <- c(fit_arima_fc$fitted, fit_arima_fc$mean) ")
#dfreal <- c(DF$y, rep(NA,h))
r("df_all <- data.frame(df_rnb,df_arima)") 
r("names(df_all) <- c('real','arima')") 

"try({names(df_all) <- c('real','arima')})\n"

In [26]:
print(r("df_all"))

try({df_all})
      real     arima
1     5500  5493.080
2     6513  6725.739
3     7602  8143.328
4     8871  8721.358
5     9828 10289.103
6    10103  9694.880
7    10064 10048.142
8     9559 10537.239
9     8881  9675.336
10    9820 10342.046
11   10494 11087.779
12   10273 10394.545
13   10131 10507.117
14    9925  9523.713
15    9306  9988.321
16    8485  9274.263
17    9289  9354.673
18   10242 10214.458
19    9815  9927.729
20    9828  9224.110
21   10052  8915.292
22    8914  9419.945
23    8647  8928.254
24    9214  8770.158
25   10192  9822.224
26   10415 10184.577
27   10421  9237.142
28    9579  9158.781
29    9202  9607.157
30    8564  8858.610
31    9942  9292.140
32   11006 10561.397
33   10734 10860.331
34   10647  9578.989
35   10139  9659.748
36    9212  9253.024
37    8400  8304.599
38    8476  9183.443
39    9544  9940.188
40   10638  9967.212
41   10006 10163.059
42    9808  9631.282
43    8813  9702.040
44    8500  9011.019
45    8433  8615.909
46   10027  9384.249

In [34]:
r("library(ggplot2)")
r("library(dplyr)")

r("df_all1 <- data.frame(dates=seq(from=(as.POSIXct(strftime('2016-01-01'))),length.out = nrow(df_all),by='days'),data = df_all)")
r("dfplot <- df_all1 %>% gather(key, value, -dates)") 

r("jpeg('real vs all models.jpg', height=4.25, width=5.5, res=200, units = 'in')")
r("p <- ggplot(dfplot, mapping = aes(x = dates, y = value, color = key) ) + geom_line() + ggtitle('real vs all models')") 
#r("return(print(p))")
print(r("p"))

try({p})
Error in try({ : object 'p' not found



In [35]:
r("write.csv(fit_arima_fc, '6_rnb_arima_pred_fc.csv')")
r("write.csv(df_all, 'df_all_arima.csv')")

"try({write.csv(df_all, 'df_all_arima.csv')})\n"